In [2]:
%cd /datalab


/datalab


In [10]:
%cd /home
import os
print(os.getcwd())

/home
/home


In [4]:
%mkdir /home/seg_train
%mkdir /home/seg_train/buildings
!git clone https://github.com/fafilia/cnn-intel_images.git
!cp -r cnn-intel_images/seg_train/buildings/* /home/seg_train/buildings/

Cloning into 'cnn-intel_images'...
remote: Enumerating objects: 24329, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 24329 (delta 2), reused 5 (delta 2), pack-reused 24323
Receiving objects: 100% (24329/24329), 351.65 MiB | 29.25 MiB/s, done.
Resolving deltas: 100% (6/6), done.
Updating files: 100% (24341/24341), done.


In [24]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau


train_dir = '/home/cnn-intel_images/seg_train'
test_dir = '/home/cnn-intel_images/seg_test'


num_epochs = 50
learning_rate = 0.001
batch_size = 64

train_transforms = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])
test_transforms = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

from torch.utils.data import random_split
train_dataset = ImageFolder(train_dir, transform=train_transforms)
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])


test_dataset = ImageFolder(test_dir, transform=test_transforms)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=1, shuffle=False)

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer4 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc1 = nn.Linear(4 * 4 * 128, 1000)
        self.fc2 = nn.Linear(1000, 512)
        self.fc3 = nn.Linear(512, 64)
        self.fc4 = nn.Linear(64, 6)
        self.drop_out = nn.Dropout(p=0.5)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc3(out)
        out = self.fc4(out)
        return out

model = ConvNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=5, verbose=True)

total_step = len(train_loader)
best_val_accuracy = 0.0
patience = 5
counter = 0
for epoch in range(num_epochs):
    model.train()
    for i, (images, labels) in enumerate(train_loader):
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{total_step}], Loss: {loss.item()}')

    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        val_accuracy = 100 * correct / total
        print(f'Epoch [{epoch + 1}/{num_epochs}] - Validation Accuracy: {val_accuracy:.2f}%')

        scheduler.step(val_accuracy)

        if val_accuracy>best_val_accuracy:
          best_val_accuracy=val_accuracy
          torch.save(model.state_dict(), 'best_convnet_model.pth')
          counter=0
        else:
          counter+=1
        if counter>=patience:
          print("Early stopping. No improvement in validation accuracy.")
          break

model.load_state_dict(torch.load('best_convnet_model.pth'))
model.eval()

test_correct=0
test_total=0
with torch.no_grad():
  for images,labels in test_loader:
    outputs=model(images)
    _, predicted = torch.max(outputs, 1)
    test_total+=labels.size(0)
    test_correct+=(predicted==labels).sum().item()
  test_accuracy = 100 * test_correct / test_total
  print(f'Test Accuracy: {test_accuracy:.2f}%')

Epoch [1/50], Step [100/176], Loss: 1.0090155601501465
Epoch [1/50] - Validation Accuracy: 64.03%
Epoch [2/50], Step [100/176], Loss: 0.8344545960426331
Epoch [2/50] - Validation Accuracy: 72.63%
Epoch [3/50], Step [100/176], Loss: 0.6796619296073914
Epoch [3/50] - Validation Accuracy: 75.73%
Epoch [4/50], Step [100/176], Loss: 0.6352912783622742
Epoch [4/50] - Validation Accuracy: 75.17%
Epoch [5/50], Step [100/176], Loss: 0.5018866062164307
Epoch [5/50] - Validation Accuracy: 77.73%
Epoch [6/50], Step [100/176], Loss: 0.8112205266952515
Epoch [6/50] - Validation Accuracy: 77.77%
Epoch [7/50], Step [100/176], Loss: 0.43437454104423523
Epoch [7/50] - Validation Accuracy: 80.00%
Epoch [8/50], Step [100/176], Loss: 0.2927994728088379
Epoch [8/50] - Validation Accuracy: 78.37%
Epoch [9/50], Step [100/176], Loss: 0.24186600744724274
Epoch [9/50] - Validation Accuracy: 79.07%
Epoch [10/50], Step [100/176], Loss: 0.23008647561073303
Epoch [10/50] - Validation Accuracy: 77.50%
Epoch [11/50], 

In [25]:

import os
from torchvision.io import read_image
from torchvision.transforms import functional as F
from PIL import Image


class UnlabeledImageFolder(torch.utils.data.Dataset):
    def __init__(self, root, transform=None):
        self.root = root
        self.transform = transform
        self.imgs = [os.path.join(root, filename) for filename in os.listdir(root) if filename.endswith(('.png', '.jpg', '.jpeg'))]

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, idx):
        img_path = self.imgs[idx]
        image = Image.open(img_path)

        if self.transform:
            image = self.transform(image)

        return image

unlabeled_dir = '/home/cnn-intel_images/seg_pred'
unlabeled_dataset = UnlabeledImageFolder(unlabeled_dir, transform=test_transforms)
unlabeled_loader = DataLoader(unlabeled_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

predictions = []

model.eval()

with torch.no_grad():
    for images in unlabeled_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        # Collect predicted labels as a list
        predicted_labels = predicted.tolist()

        predictions.extend(predicted_labels)

print("Predictions for unlabeled images:", predictions)


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Predictions for unlabeled images: [1, 2, 5, 2, 1, 0, 0, 0, 4, 3, 2, 1, 2, 2, 4, 4, 2, 4, 3, 4, 2, 3, 1, 1, 0, 3, 3, 3, 3, 3, 1, 0, 1, 0, 2, 4, 1, 0, 3, 3, 2, 1, 1, 0, 4, 4, 5, 4, 3, 4, 0, 3, 0, 3, 4, 3, 2, 2, 5, 2, 3, 4, 0, 1, 2, 3, 0, 4, 0, 4, 1, 0, 2, 3, 0, 4, 0, 1, 3, 5, 3, 2, 3, 0, 4, 3, 1, 5, 0, 0, 2, 4, 4, 2, 4, 1, 5, 1, 4, 0, 3, 5, 5, 0, 2, 1, 5, 0, 4, 0, 3, 1, 1, 4, 2, 1, 1, 0, 2, 5, 0, 4, 0, 4, 4, 2, 2, 2, 5, 0, 2, 5, 1, 3, 4, 4, 1, 1, 3, 1, 5, 2, 5, 1, 1, 0, 5, 3, 5, 5, 1, 1, 2, 1, 5, 1, 2, 2, 0, 3, 1, 3, 3, 3, 2, 5, 2, 0, 0, 0, 4, 3, 5, 1, 3, 4, 0, 0, 0, 2, 2, 1, 2, 1, 3, 5, 4, 0, 3, 5, 1, 2, 2, 0, 1, 3, 2, 3, 5, 0, 3, 5, 3, 0, 3, 3, 3, 2, 3, 4, 3, 1, 5, 1, 4, 0, 1, 0, 3, 5, 4, 5, 3, 0, 4, 5, 1, 3, 2, 4, 4, 1, 0, 3, 2, 2, 3, 2, 0, 5, 0, 2, 4, 3, 3, 3, 1, 4, 1, 5, 2, 5, 0, 4, 4, 0, 3, 2, 1, 3, 5, 3, 1, 4, 3, 1, 4, 3, 0, 2, 2, 3, 1, 1, 5, 1, 2, 4, 1, 1, 5, 5, 2, 5, 4, 1, 4, 1, 4, 4, 3, 4, 2, 2, 4, 4, 0, 0, 5, 0, 3, 2, 1, 2, 3, 4, 2, 4, 3, 0, 0, 5, 1, 3, 3, 3, 2, 0, 4, 2, 5, 0,

In [26]:
print("Predicted class probabilities:", outputs)
print("Predicted class indices:", predicted)

Predicted class probabilities: tensor([[-2.3927, -0.6151,  2.7273,  3.8996, -1.8803, -4.8344],
        [ 4.7265, -0.9829, -2.9880, -1.8491,  0.5012,  2.8823],
        [-1.1536,  4.7541, -0.1365, -3.1326, -2.7757,  4.0931],
        [-5.7885, -1.5433,  3.4273, 11.0009, -5.3236, -6.3051],
        [-2.6093, -2.0571,  2.7254,  4.4121,  0.3595, -5.9892]])
Predicted class indices: tensor([3, 0, 1, 3, 3])


In [28]:
from random import randint
labels=["buildings", "forest", "glacier", "mountain", "sea", "street"]

for _ in range(100):
  random_index=randint(1,1000)
  image=test_dataset[random_index][0];label=test_dataset[random_index][1]


  model.eval()
  with torch.no_grad():
    predictions=model(image.unsqueeze(0))

  _, predicted=torch.max(predictions,1)

  print("True:",labels[label],"Predicted:",labels[predicted])

True: buildings Predicted: buildings
True: buildings Predicted: buildings
True: forest Predicted: forest
True: buildings Predicted: buildings
True: glacier Predicted: glacier
True: buildings Predicted: street
True: buildings Predicted: buildings
True: buildings Predicted: forest
True: buildings Predicted: mountain
True: buildings Predicted: buildings
True: forest Predicted: forest
True: buildings Predicted: street
True: buildings Predicted: buildings
True: buildings Predicted: buildings
True: buildings Predicted: buildings
True: glacier Predicted: glacier
True: buildings Predicted: buildings
True: forest Predicted: forest
True: forest Predicted: forest
True: forest Predicted: forest
True: buildings Predicted: buildings
True: forest Predicted: forest
True: buildings Predicted: buildings
True: buildings Predicted: buildings
True: forest Predicted: forest
True: forest Predicted: forest
True: forest Predicted: forest
True: buildings Predicted: buildings
True: glacier Predicted: mountain
Tr